- This is a practice notebook. It differs slightly from the original implementation of the author, in that in exercise e), I added alpha dropout layer after every hidden layer but the author seem to add only at the end
- Similarly, I didn't change the alpha dropout layer to MCdropout layer while predition
- Same for f) as well.

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import tensorflow as tf
from tensorflow import keras

2023-07-08 10:46:47.281666: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-08 10:46:47.303412: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-08 10:46:47.694486: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Setting up a virtual GPU

In [3]:
# Creating a virtual GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict Tensorflow to allocate only 1 GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(
            gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=3072)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical Devices", len(logical_gpus), "Logical Devices")
    except RuntimError as e:
        print(e)
        

1 Physical Devices 1 Logical Devices


2023-07-08 10:46:48.502323: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-08 10:46:48.523865: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-08 10:46:48.523929: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
gpus = tf.config.list_physical_devices('GPU')
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### Problem Statement

Practice training a Deep Neural Network on the CIFAR10 image Dataset

- a) Build a DNN with 20 hiddden layers of 100 neuron each. Use He initialization and the ELU activation
- b) Using Nadam optimization and early stopping, train the network on the CIFAR10 dataset. You can load it with keras.datasets.cifar10.load-data(). The Dataset is composed of 60000 32 * 32-pixel color images(50,000 for training and 10000 for testing) with 10 classes, so you'll need a softmax output layer with 10 neurons. Remember to search for the right learning rate each time you change the model's architecture or hyperparameters.
- c) Now try adding Batch Normalization and compare the learning curves: Is it converging faster than before? Does it producre a better model? How does it affect the training speed?
- d) Try replacing Batch Normalization with SELU and make the necessary adjustments to ensure the network self-normalzation(i.e., standardize the intput features, use LeCun normal initialization, make sure the DNN contains only a sequence of dense layers, etc.).
- e) Try regularizing the model with alpha dropout. Then, without retraining your model, see if you can achieve better accuracy using MC Dropout.
- f) Retrain your model using Icycle scheduling and see if it improves training speed and model accuracy. 

### a)

In [5]:
(X_train_full, Y_train_full), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [6]:
x_train, x_valid = X_train_full[:45000]/255., X_train_full[45000:]/255.
y_train, y_valid = Y_train_full[:45000], Y_train_full[45000:]

In [7]:
X_train_full.shape

(50000, 32, 32, 3)

In [8]:
model = keras.models.Sequential([keras.layers.Flatten(input_shape=(32, 32, 3)),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.Dense(units=10, activation="softmax")])

### b

In [9]:
early_stopping = keras.callbacks.EarlyStopping(patience=10)
model_cp = keras.callbacks.ModelCheckpoint("my_cifar10_model.h5")
model.compile(loss = "sparse_categorical_crossentropy", 
              optimizer = keras.optimizers.experimental.Nadam(learning_rate=1e-3, beta_1=0.9, beta_2=0.999),
              metrics=["accuracy"])

In [10]:
import time

start = time.time()
model.fit(x_train, y_train, epochs=100,
         validation_data=(x_valid, y_valid),
         callbacks=[early_stopping, model_cp])
end = time.time()
print("Time taken for the training: ",end-start)

Epoch 1/100


2023-07-08 10:46:51.644260: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-07-08 10:46:51.645959: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7fce14ff9f70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-07-08 10:46:51.645971: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2023-07-08 10:46:51.649723: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-07-08 10:46:51.721058: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8700
2023-07-08 10:46:51.787278: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the p

1407/1407 [==============================] - 9s 3ms/step - loss: 2.0478 - accuracy: 0.2431 - val_loss: 1.9764 - val_accuracy: 0.2692
Epoch 2/100
1407/1407 [==============================] - 4s 3ms/step - loss: 1.8416 - accuracy: 0.3270 - val_loss: 1.8858 - val_accuracy: 0.3070
Epoch 3/100
1407/1407 [==============================] - 4s 3ms/step - loss: 1.7749 - accuracy: 0.3568 - val_loss: 1.7629 - val_accuracy: 0.3508
Epoch 4/100
1407/1407 [==============================] - 4s 3ms/step - loss: 1.7215 - accuracy: 0.3805 - val_loss: 1.7465 - val_accuracy: 0.3868
Epoch 5/100
1407/1407 [==============================] - 4s 3ms/step - loss: 1.6847 - accuracy: 0.3946 - val_loss: 1.7451 - val_accuracy: 0.3876
Epoch 6/100
1407/1407 [==============================] - 4s 3ms/step - loss: 1.6626 - accuracy: 0.4068 - val_loss: 1.7323 - val_accuracy: 0.3818
Epoch 7/100
1407/1407 [==============================] - 4s 3ms/step - loss: 1.6302 - accuracy: 0.4181 - val_loss: 1.6424 - val_accuracy: 0.42

Time taken for the training:  102.81397342681885


In [11]:
model = keras.models.load_model("my_cifar10_model.h5")
model.evaluate(x_valid, y_valid)

157/157 [==============================] - 0s 881us/step - loss: 1.6271 - accuracy: 0.4138


[1.62712824344635, 0.4138000011444092]

### c)

In [12]:
model_batch_normalization = keras.models.Sequential([keras.layers.Flatten(input_shape=(32, 32, 3)),
                                keras.layers.BatchNormalization(),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.BatchNormalization(),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.BatchNormalization(),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.BatchNormalization(),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.BatchNormalization(),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.BatchNormalization(),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.BatchNormalization(),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.BatchNormalization(),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.BatchNormalization(),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.BatchNormalization(),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.BatchNormalization(),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.BatchNormalization(),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.BatchNormalization(),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.BatchNormalization(),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.BatchNormalization(),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.BatchNormalization(),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.BatchNormalization(),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.BatchNormalization(),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.BatchNormalization(),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.BatchNormalization(),
                                keras.layers.Dense(units=100, activation="elu", kernel_initializer="he_normal"),
                                keras.layers.BatchNormalization(),
                                keras.layers.Dense(units=10, activation="softmax")])

In [13]:
early_stopping = keras.callbacks.EarlyStopping(patience=10)
model_batch_normalization.compile(loss = "sparse_categorical_crossentropy", 
              optimizer = keras.optimizers.experimental.Nadam(learning_rate=1e-3, beta_1=0.9, beta_2=0.999),
              metrics = ["accuracy"])

In [14]:
start_BN = time.time()
model_batch_normalization.fit(x_train, y_train, epochs=100,
                             validation_data=(x_valid, y_valid),
                             callbacks=[early_stopping, model_cp])
end_BN = time.time()

print("Time taken to trian the model with BN: ", end_BN-start_BN)

Epoch 1/100
1407/1407 [==============================] - 18s 7ms/step - loss: 1.9009 - accuracy: 0.3152 - val_loss: 1.6930 - val_accuracy: 0.3968
Epoch 2/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.7168 - accuracy: 0.3879 - val_loss: 1.6174 - val_accuracy: 0.4258
Epoch 3/100
1407/1407 [==============================] - 9s 7ms/step - loss: 1.6534 - accuracy: 0.4134 - val_loss: 1.5491 - val_accuracy: 0.4476
Epoch 4/100
1407/1407 [==============================] - 9s 7ms/step - loss: 1.5968 - accuracy: 0.4371 - val_loss: 1.5338 - val_accuracy: 0.4618
Epoch 5/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.5509 - accuracy: 0.4497 - val_loss: 1.4596 - val_accuracy: 0.4874
Epoch 6/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.5149 - accuracy: 0.4654 - val_loss: 1.4285 - val_accuracy: 0.5016
Epoch 7/100
1407/1407 [==============================] - 9s 7ms/step - loss: 1.4817 - accuracy: 0.4782 - val_loss: 1.4596 - va

Time taken to trian the model with BN:  237.58348441123962


In [15]:
model = keras.models.load_model("my_cifar10_model.h5")
model.evaluate(x_valid, y_valid)

157/157 [==============================] - 0s 1ms/step - loss: 1.3532 - accuracy: 0.5302


[1.3532238006591797, 0.5302000045776367]

### d)

In [16]:
# !pip install scikit-learn

In [17]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

training_samples, _, _, _ = x_train.shape
valid_samples, _ , _, _ = x_valid.shape

x_train_reshaped = x_train.reshape(training_samples, -1)
x_valid_reshaped = x_valid.reshape(valid_samples, -1)

x_train_scaled = scaler.fit_transform(x_train_reshaped)
x_valid_scaled = scaler.fit_transform(x_valid_reshaped)

x_train_standardized = x_train_scaled.reshape(x_train.shape)
x_valid_standardized = x_valid_scaled.reshape(x_valid.shape)

In [18]:
model_selu = keras.models.Sequential([keras.layers.Flatten(input_shape=(32, 32, 3)),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                keras.layers.Dense(units=10, activation="softmax")])

In [19]:
model_selu.compile(loss = "sparse_categorical_crossentropy",
                   optimizer=keras.optimizers.experimental.Nadam(learning_rate=1e-3, beta_1=0.9, beta_2=0.999),
                  metrics=["accuracy"])

In [20]:
import time 
start = time.time()
early_stopping = keras.callbacks.EarlyStopping(patience=10)
model_selu.fit(x_train_standardized, y_train, epochs=100,
              validation_data=(x_valid_standardized, y_valid),
              callbacks=[early_stopping, model_cp])
end = time.time()
print("Time taken for training the model_selu: ", end-start)

Epoch 1/100
1407/1407 [==============================] - 7s 3ms/step - loss: 1.9511 - accuracy: 0.2873 - val_loss: 1.8829 - val_accuracy: 0.3028
Epoch 2/100
1407/1407 [==============================] - 4s 3ms/step - loss: 1.7697 - accuracy: 0.3529 - val_loss: 1.7943 - val_accuracy: 0.3806
Epoch 3/100
1407/1407 [==============================] - 4s 3ms/step - loss: 1.6766 - accuracy: 0.3949 - val_loss: 1.6495 - val_accuracy: 0.4146
Epoch 4/100
1407/1407 [==============================] - 4s 3ms/step - loss: 1.6178 - accuracy: 0.4184 - val_loss: 1.6445 - val_accuracy: 0.4202
Epoch 5/100
1407/1407 [==============================] - 4s 3ms/step - loss: 1.5721 - accuracy: 0.4391 - val_loss: 1.6712 - val_accuracy: 0.4232
Epoch 6/100
1407/1407 [==============================] - 4s 3ms/step - loss: 1.5316 - accuracy: 0.4548 - val_loss: 1.5965 - val_accuracy: 0.4332
Epoch 7/100
1407/1407 [==============================] - 4s 3ms/step - loss: 1.6408 - accuracy: 0.4034 - val_loss: 1.6715 - val_ac

Time taken for training the model_selu:  100.98614358901978


In [21]:
model = keras.models.load_model("my_cifar10_model.h5")
model.evaluate(x_valid_standardized, y_valid)

157/157 [==============================] - 0s 925us/step - loss: 1.4906 - accuracy: 0.4896


[1.4906381368637085, 0.4896000027656555]

### e) Alpha dropout

In [22]:
model_alpha_dropout = model_selu = keras.models.Sequential([keras.layers.Flatten(input_shape=(32, 32, 3)),
                                                            keras.layers.AlphaDropout(rate=0.2),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                                            keras.layers.AlphaDropout(rate=0.2),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                                            keras.layers.AlphaDropout(rate=0.2),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                                            keras.layers.AlphaDropout(rate=0.2),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                                            keras.layers.AlphaDropout(rate=0.2),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                                            keras.layers.AlphaDropout(rate=0.2),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                                            keras.layers.AlphaDropout(rate=0.2),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                                            keras.layers.AlphaDropout(rate=0.2),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                                            keras.layers.AlphaDropout(rate=0.2),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                                            keras.layers.AlphaDropout(rate=0.2),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                                            keras.layers.AlphaDropout(rate=0.2),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                                            keras.layers.AlphaDropout(rate=0.2),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                                            keras.layers.AlphaDropout(rate=0.2),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                                            keras.layers.AlphaDropout(rate=0.2),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                                            keras.layers.AlphaDropout(rate=0.2),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                                            keras.layers.AlphaDropout(rate=0.2),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                                            keras.layers.AlphaDropout(rate=0.2),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                                            keras.layers.AlphaDropout(rate=0.2),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                                            keras.layers.AlphaDropout(rate=0.2),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                                            keras.layers.AlphaDropout(rate=0.2),
                                keras.layers.Dense(units=100, activation="selu", kernel_initializer="lecun_normal"),
                                                            keras.layers.AlphaDropout(rate=0.2),
                                keras.layers.Dense(units=10, activation="softmax")])

In [23]:
model_alpha_dropout.compile(loss = "sparse_categorical_crossentropy",
                            optimizer = "nadam",
                            metrics=["accuracy"])

In [24]:
start_AD = time.time()
model_alpha_dropout.fit(x_train_standardized, y_train, epochs=100,
                       validation_data=(x_valid_standardized, y_valid),
                       callbacks=[early_stopping, model_cp])
end_AD = time.time()
print("The time taken to train model_AD: ", end_AD-start_AD)

Epoch 1/100
1407/1407 [==============================] - 9s 4ms/step - loss: 2.2930 - accuracy: 0.1466 - val_loss: 178.8407 - val_accuracy: 0.1392
Epoch 2/100
1407/1407 [==============================] - 5s 3ms/step - loss: 2.0669 - accuracy: 0.1852 - val_loss: 120.5056 - val_accuracy: 0.1428
Epoch 3/100
1407/1407 [==============================] - 5s 4ms/step - loss: 2.0544 - accuracy: 0.1920 - val_loss: 73.8143 - val_accuracy: 0.1240
Epoch 4/100
1407/1407 [==============================] - 5s 4ms/step - loss: 2.0269 - accuracy: 0.2089 - val_loss: 111.7378 - val_accuracy: 0.1220
Epoch 5/100
1407/1407 [==============================] - 5s 3ms/step - loss: 2.0265 - accuracy: 0.2131 - val_loss: 80.2081 - val_accuracy: 0.1486
Epoch 6/100
1407/1407 [==============================] - 5s 3ms/step - loss: 2.0187 - accuracy: 0.2230 - val_loss: 93.5622 - val_accuracy: 0.1972
Epoch 7/100
1407/1407 [==============================] - 5s 3ms/step - loss: 2.0100 - accuracy: 0.2356 - val_loss: 101.92

The time taken to train model_AD:  125.67989873886108


In [25]:
model = keras.models.load_model("my_cifar10_model.h5")
model.evaluate(x_valid_standardized, y_valid)

157/157 [==============================] - 0s 977us/step - loss: 60.0472 - accuracy: 0.1714


[60.04716873168945, 0.17139999568462372]

In [26]:
import numpy as np

y_probas = np.stack([model_alpha_dropout(x_valid_standardized, training=True)
                     for sample in range(100)])

y_probas = y_probas.mean(axis=0)

In [27]:
np.round(model_alpha_dropout.predict(x_valid_standardized[:1]), 2)

1/1 [==============================] - 0s 76ms/step


array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]], dtype=float32)

In [28]:
np.round(y_probas[:1], 2)

array([[0.02, 0.02, 0.15, 0.08, 0.23, 0.11, 0.23, 0.15, 0.01, 0.01]],
      dtype=float32)

In [29]:
np.round(y_probas[:], 2)

array([[0.02, 0.02, 0.15, ..., 0.15, 0.01, 0.01],
       [0.12, 0.29, 0.03, ..., 0.04, 0.17, 0.23],
       [0.02, 0.01, 0.17, ..., 0.13, 0.01, 0.01],
       ...,
       [0.17, 0.21, 0.04, ..., 0.04, 0.25, 0.18],
       [0.3 , 0.08, 0.06, ..., 0.02, 0.39, 0.08],
       [0.05, 0.08, 0.1 , ..., 0.18, 0.04, 0.06]], dtype=float32)

In [30]:
y_pred = np.argmax(y_probas, axis = 1)

y_pred = y_pred.reshape(y_valid.shape)

accuracy = np.sum(y_pred == y_valid) / len(y_valid)
accuracy

0.2714

### f)

In [31]:
K = keras.backend
class OneCycleScheduler(keras.callbacks.Callback):
    def __init__(self, iterations, max_rate, start_rate=None, last_iteration=None, last_rate=None):
        self.iterations = iterations
        self.max_rate = max_rate
        self.start_rate = start_rate or max_rate / 10
        self.last_iterations = last_iteration or iterations //10+1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_rate = last_rate or self.start_rate / 1000
        self.iteration = 0
        
    def _interpolate(self, iter1, iter2, rate1, rate2):
        return ((rate2-rate1) * (self.iteration-iter1) / (iter2-iter1) + rate1)
    
    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            rate = self._interpolate(0, self.half_iteration, self.start_rate, self.max_rate)
        elif self.iteration < 2 * self.half_iteration:
            rate = self._interpolate(self.half_iteration, 2 * self.half_iteration,
                                     self.max_rate, self.start_rate)
        else:
            rate = self._interpolate(2 * self.half_iteration, self.iterations,
                                     self.start_rate, self.last_rate)
        self.iteration += 1
        K.set_value(self.model.optimizer.learning_rate, rate)

In [32]:
import math

In [33]:
n_epochs = 25
batch_size = 128
onecycle = OneCycleScheduler(iterations = (math.ceil(len(x_train)/batch_size) * n_epochs), max_rate = 0.05)
start_1cy = time.time()
history = model_alpha_dropout.fit(x_train_standardized, y_train, epochs=n_epochs, batch_size=batch_size, 
                             validation_data=(x_valid_standardized, y_valid),
                             callbacks=[onecycle, early_stopping, model_cp])
end_1cy = time.time()
print("The time taken to train model_alpha_dropout using onecycle learning scheduler: ", end_1cy-start_1cy)

Epoch 1/25
352/352 [==============================] - 1s 4ms/step - loss: 2.2573 - accuracy: 0.1875 - val_loss: 2.3330 - val_accuracy: 0.0970
Epoch 2/25
352/352 [==============================] - 1s 4ms/step - loss: 35.8995 - accuracy: 0.0996 - val_loss: 6.5997 - val_accuracy: 0.1064
Epoch 3/25
352/352 [==============================] - 1s 4ms/step - loss: 4.1982 - accuracy: 0.0996 - val_loss: 2.4267 - val_accuracy: 0.0970
Epoch 4/25
352/352 [==============================] - 1s 4ms/step - loss: 357225920.0000 - accuracy: 0.0998 - val_loss: 29.8931 - val_accuracy: 0.0958
Epoch 5/25
352/352 [==============================] - 1s 4ms/step - loss: 17134095695872.0000 - accuracy: 0.1003 - val_loss: 38.4913 - val_accuracy: 0.0958
Epoch 6/25
352/352 [==============================] - 1s 4ms/step - loss: 89721866813440.0000 - accuracy: 0.0977 - val_loss: 22.3636 - val_accuracy: 0.0950
Epoch 7/25
352/352 [==============================] - 1s 4ms/step - loss: 6040175104.0000 - accuracy: 0.1001 -

In [34]:
model = keras.models.load_model("my_cifar10_model.h5")
model.evaluate(x_valid_standardized, y_valid)

157/157 [==============================] - 0s 852us/step - loss: nan - accuracy: 0.0986


[nan, 0.09860000014305115]